In [124]:
import pandas as pd
import numpy as np

In [320]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

In [321]:
# 나이 누락 -> 호칭에 따른 평균 나이로 치환하는 함수
def name2age(data):
    
    #모든 호칭 데이터 추출해 저장
    dfName=data.Name.str.extract(' ([A-Z][a-z]+)[.]')
    
    #각 호칭 별 인덱스 저장
    MrIdx=dfName.loc[dfName.loc[:,0]=='Mr'].index
    MrsIdx=dfName.loc[dfName.loc[:,0]=='Mrs'].index
    MissIdx=dfName.loc[dfName.loc[:,0]=='Miss'].index
    MasterIdx=dfName.loc[dfName.loc[:,0]=='Master'].index
    DrIdx=dfName.loc[dfName.loc[:,0]=='Dr'].index
    
    # 호칭 별 평균 나이 저장
    MrAge=data.loc[MrIdx]['Age'].mean()
    MrsAge=data.loc[MrsIdx]['Age'].mean()
    MissAge=data.loc[MissIdx]['Age'].mean()
    MasterAge=data.loc[MasterIdx]['Age'].mean()
    DrAge=data.loc[DrIdx]['Age'].mean()
    
    # 나이 누락된 데이터 추출
    dfAgeNull=data.loc[data.Age.isnull()]
    
    # 호칭에 따라 나이 누락된 데이터에 평균 나이 삽입
    for i in dfAgeNull.index:
        name=dfName.loc[i,0]
        if name=='Mr':
            data.loc[i,'Age']=MrAge
        elif name=='Mrs':
            data.loc[i,'Age']=MrsAge
        elif name=='Miss':
            data.loc[i,'Age']=MissAge
        elif name=='Master':
            data.loc[i,'Age']=MasterAge
        else: data.loc[i,'Age']=DrAge
        
    return(data)

In [322]:
name2age(train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.773973,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C


In [451]:
# FamilySize 열 추가하는 함수
def famSize(data):
    fsize=data.SibSp+data.Parch
    data['FamilySize']=fsize
    
    # np.where 활용
    data['FamilySize_wh']=np.where(fsize==0,'S',(np.where(fsize==1,'M',
    (np.where(fsize>1,'L',fsize)))))
    
    # cut 활용
    bins=[fsize.min()-1,1,2,fsize.max()+1]
    labels=['S','M','L']
    cut=pd.cut(fsize,bins,labels=labels,right=False)
    data['FamilySize_cut']=cut
    
    return data

In [452]:
famSize(train).FamilySize_cut.value_counts()

S    537
L    193
M    161
Name: FamilySize_cut, dtype: int64

In [455]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [471]:
# 순위 올리기
test['Survived']=(((test.Sex=='female') & (test.Age>0)) | 
((test.Sex=='male') &(test.FamilySize>=1))).astype(int)

sub=test[['PassengerId','Survived']]
sub.to_csv('sub.csv',index=False)